In [3]:
# Say you have a sequence of numbers, and you want to find its longest increasing subsequence - or one of them,
# if there are more.
from itertools import combinations

def naive_lis(seq):
    '''A Naive Solution to the Longest Increasing Subsequence Problem'''
    for length in range(len(seq), 0, -1):      # n, n-1, ..., 1
        for sub in combinations(seq, length):  # Subsequences of given length
            if list(sub) == sorted(sub):       # An increasing subsequence?
                return sub                     # Return it!

In [4]:
naive_lis([3,1,0,2,4])

(1, 2, 4)

In [6]:
# Don't Repeat Yourself
# The basic idea of this chapter is to avoid (vermeiden) having your 'algorithm' repeat itself.
# The principle is so simple, and even really easy to implement (at least (geringer) in Python),
# but the mojo (Glücksbringer) here is really deep (tiefgründig), as you'll see as we progress.
def fib(i):
    if i < 2: return 1
    return fib(i-1) + fib(i-2)


from functools import wraps

def memo(func):
    '''A Memoizing Decorator. The idea of memoized function is that it caches its return values. If you call it a
    second time with the same parameters, it will simply return the cached value. You can certainly (allerdings) 
    put this sort of caching logic inside your function, but memo function is a more reusable (Mehrweg-) solution.
    '''
    cache = {}                           # Stored subproblem solutions
    @wraps(func)                         # Make wrap look like func
    def wrap(*args):                     # The memoized wrapper
        if args not in cache:            # Not already computed?
            cache[args] = func(*args)    # Compute & cache the solution
        return cache[args]               # Return the cached solution
    return wrap                          # Return the wrapper

fib = memo(fib)

In [10]:
fib(100)

573147844013817084101L

In [12]:
# It's even designed to be used as a decorator:
@memo
def fib(i):
    if i < 2: return 1
    return fib(i-1) + fib(i-2)

fib(100)

573147844013817084101L

In [15]:
# overlapping subproblems
# a recursion formulation of the powers of two.
@memo # Option 1
def two_pow(i):
    if i == 0: return 1
    return two_pow(i-1) + two_pow(i-1)

print(two_pow(10))
print(two_pow(100))

1024
1267650600228229401496703205376


In [17]:
# overlapping subproblems
# a recursion formulation of the powers of two.
# Option 2
def two_pow(i):
    if i == 0: return 1
    return 2 * two_pow(i-1)

print(two_pow(10))
print(two_pow(100))

1024
1267650600228229401496703205376


In [19]:
# Calculating binomial coefficients (Pascal's triangle)
# We decompose the problem by conditioning (Aufbereitung) on whether some element is included.
# C(n,0) = 1 for the single empty subset, and C(0,k) = 0, k > 0, for nonempty subsets of an empty set.
# path counting (abzählen)

@memo
def C(n,k):
    if k == 0: return 1
    if n == 0: return 0
    return C(n-1,k-1) + C(n-1,k)

print(C(4,2))
print(C(10,7))
print(C(100,50))

6
120
100891344545564193334812497256


In [20]:
from collections import defaultdict

n, k = 10, 7
C = defaultdict(int)
for row in range(n+1):
    C[row, 0] = 1
    for col in range(1,k+1):
        C[row,col] = C[row-1,col-1] + C[row-1,col]
        
C[n,k]

120